In [1]:
import numpy as np
import pandas as pd
import sklearn

from sklearn import metrics
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier

cтильный аутпут

In [2]:
from IPython.core.display import HTML

HTML(
    r"""
<style>
    .output-plaintext, .output-stream, .output {
        font-family: "JetBrainsMono Nerd Font Mono"; # Any monospaced font should work
    }
</style>
"""
)

np.set_printoptions(precision=2)
pd.set_option("display.precision", 2)

In [3]:
seed = 42
size = 1_000

In [4]:
train_df = pd.read_csv("./data/prepared/train.csv", low_memory=False).drop(
    columns=["id"]
)
test_df = pd.read_csv("./data/prepared/test.csv", low_memory=False).drop(columns=["id"])
train_labels_df = pd.read_csv(
    "./data/prepared/train_labels.csv", low_memory=False
).drop(columns=["id"])

In [5]:
train_df

,release,n_0000,n_0001,n_0002,n_0003,n_0004,n_0005,n_0006,n_0007,n_0009,...,c_1366,c_1367,c_1369,c_1370,c_1372,c_1373,c_1374,c_1375,c_1376,c_1377
0,a,NaN,NaN,0.03,NaN,NaN,0.37,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,a,NaN,q,NaN,NaN,NaN
1,a,NaN,NaN,0.03,NaN,NaN,0.32,NaN,NaN,NaN,...,NaN,NaN,NaN,a,a,NaN,NaN,NaN,NaN,NaN
2,a,NaN,NaN,0.02,NaN,NaN,0.34,NaN,NaN,NaN,...,NaN,NaN,NaN,a,a,NaN,b,NaN,NaN,NaN
3,a,NaN,NaN,0.04,NaN,NaN,0.45,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,a,NaN,NaN,NaN,NaN,NaN
4,c,NaN,NaN,0.04,NaN,NaN,0.32,NaN,NaN,NaN,...,NaN,NaN,NaN,b,a,NaN,a,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7995,a,NaN,NaN,0.05,NaN,NaN,0.39,NaN,NaN,NaN,...,a,NaN,NaN,NaN,a,NaN,NaN,NaN,NaN,b
7996,a,NaN,NaN,0.03,NaN,NaN,0.39,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,a,NaN,a,NaN,NaN,NaN
7997,c,NaN,0.90,0.03,NaN,NaN,0.39,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,a,NaN,NaN,NaN,NaN,NaN
7998,c,NaN,NaN,0.05,NaN,NaN,0.29,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,a,NaN,q,NaN,NaN,NaN


In [6]:
train_labels_df

,service_a,service_b,service_c,service_d,service_e,service_f,service_g,service_h,service_i,service_j,service_k,service_l,service_m,service_n
0,1,1,0,0,0,0,0,0,0,1,1,0,0,0
1,0,0,0,0,0,0,0,0,0,1,1,0,0,0
2,0,0,0,0,0,0,0,0,0,1,1,0,0,0
3,0,0,0,1,0,0,0,0,0,1,0,1,0,0
4,0,0,0,1,1,0,0,0,0,0,1,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7995,1,1,1,0,0,0,1,0,0,1,1,1,1,1
7996,0,0,0,0,0,0,0,0,0,1,1,0,0,0
7997,1,0,0,0,0,0,0,0,0,1,1,0,0,0
7998,1,1,1,0,0,0,0,0,0,1,1,0,0,0


## Baseline: взять фичи, которые определены на ~90% процентах семплов и на них обучаться.логрегом, деревом..

In [7]:
train_counts = train_df.describe(include="all").loc["count"]
well_done_df = train_df[train_counts[train_counts > 7000].index]

well_done_df

,release,n_0002,n_0005,n_0019,n_0038,n_0047,n_0050,n_0052,n_0061,n_0067,...,c_1223,c_1227,c_1236,c_1244,c_1252,c_1259,c_1286,c_1316,c_1348,c_1372
0,a,0.03,0.37,0.0,0.19,1,1,1,1,0.93,...,c,a,c,d,b,n,b,b,b,a
1,a,0.03,0.32,0.0,0.18,1,1,1,1,0.93,...,a,a,c,d,b,e,b,b,b,a
2,a,0.02,0.34,0.0,0.29,1,1,1,1,0.43,...,c,a,a,d,b,w,b,b,b,a
3,a,0.04,0.45,0.0,0.37,1,1,1,1,0.57,...,c,a,c,d,b,e,b,a,b,a
4,c,0.04,0.32,0.0,0.18,1,1,1,1,0.93,...,c,a,c,d,b,e,b,b,b,a
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7995,a,0.05,0.39,0.0,0.21,1,1,1,1,0.00,...,c,b,c,a,b,i,a,a,c,a
7996,a,0.03,0.39,0.0,0.29,1,1,1,1,0.71,...,c,a,c,d,b,w,b,b,b,a
7997,c,0.03,0.39,0.0,0.18,1,1,1,1,1.00,...,c,a,c,d,b,n,b,b,b,a
7998,c,0.05,0.29,0.2,0.18,1,1,1,1,0.36,...,c,a,a,a,b,n,b,b,b,a


### Для logreg, trees нужно заполнить наны, а также препроцессить категориальные фичи

Сделаем просто:
* ohe_hot для категориальных фичей → nan-ы превратятся в [0, 0, ..., 0]
* mean для численных фичей

In [8]:
# convert categorical. na converts to [0, 0, ..., 0]
well_done_df = pd.get_dummies(well_done_df)

# impute nans for numerical features
well_done_df = pd.DataFrame(
    columns=well_done_df.columns,
    data=SimpleImputer(strategy="mean").fit_transform(well_done_df),
)

well_done_df

,n_0002,n_0005,n_0019,n_0038,n_0047,n_0050,n_0052,n_0061,n_0067,n_0075,...,c_1286_a,c_1286_b,c_1316_a,c_1316_b,c_1348_a,c_1348_b,c_1348_c,c_1348_d,c_1372_a,c_1372_b
0,0.03,0.37,0.0,0.19,1.0,1.0,1.0,1.0,0.93,1.0,...,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0
1,0.03,0.32,0.0,0.18,1.0,1.0,1.0,1.0,0.93,1.0,...,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0
2,0.02,0.34,0.0,0.29,1.0,1.0,1.0,1.0,0.43,1.0,...,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0
3,0.04,0.45,0.0,0.37,1.0,1.0,1.0,1.0,0.57,1.0,...,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
4,0.04,0.32,0.0,0.18,1.0,1.0,1.0,1.0,0.93,1.0,...,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7995,0.05,0.39,0.0,0.21,1.0,1.0,1.0,1.0,0.00,1.0,...,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
7996,0.03,0.39,0.0,0.29,1.0,1.0,1.0,1.0,0.71,1.0,...,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0
7997,0.03,0.39,0.0,0.18,1.0,1.0,1.0,1.0,1.00,1.0,...,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0
7998,0.05,0.29,0.2,0.18,1.0,1.0,1.0,1.0,0.36,1.0,...,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0


### TODO: Imputer нужно обучать только на трейне...

In [9]:
train_features, valid_features, train_targets, valid_targets = train_test_split(
    well_done_df, train_labels_df, random_state=seed, test_size=0.3
)

#### Обучаем

In [10]:
cross_val_score(
    RandomForestClassifier(random_state=seed),
    well_done_df,
    train_labels_df,
    cv=5,
    scoring="roc_auc",
)

array([0.79, 0.8 , 0.8 , 0.79, 0.79])

In [11]:
cross_val_score(
    MultiOutputClassifier(LogisticRegression(solver="liblinear", random_state=seed)),
    well_done_df,
    train_labels_df,
    cv=5,
    scoring="roc_auc",
)

array([0.81, 0.81, 0.8 , 0.8 , 0.82])

#### Посмотрим подробнее на метрики по разными таргетам

In [12]:
# с параметрами играться не будем
clf = RandomForestClassifier(
    random_state=seed,
)
clf.fit(train_features, train_targets)
valid_preds = clf.predict_proba(valid_features)
valid_preds = np.transpose([y_pred[:, 1] for y_pred in valid_preds])

rf_metrics = pd.Series(
    index=train_labels_df.columns,
    data=metrics.roc_auc_score(
        valid_targets,
        valid_preds,
        average=None,
    ),
)

In [13]:
importances = pd.Series(index=well_done_df.columns, data=clf.feature_importances_)

importances.sort_values(ascending=False)[:10]

c_1259_n    0.03
n_0002      0.03
n_0078      0.03
n_0108      0.03
o_0176      0.02
n_0038      0.02
n_0067      0.02
n_0005      0.02
o_0264      0.02
o_0144      0.01
dtype: float64

In [14]:
train_features, valid_features, train_targets, valid_targets = train_test_split(
    well_done_df, train_labels_df, random_state=seed, test_size=0.3
)

inner_clf = LogisticRegression(solver="liblinear", random_state=seed)
clf = MultiOutputClassifier(inner_clf).fit(train_features, train_targets)
valid_preds = clf.predict_proba(valid_features)
valid_preds = np.transpose([y_pred[:, 1] for y_pred in valid_preds])

lr_metrics = pd.Series(
    index=train_labels_df.columns,
    data=metrics.roc_auc_score(
        valid_targets,
        valid_preds,
        average=None,
    ),
)

In [15]:
pd.DataFrame({"log_reg": lr_metrics, "random_forest": rf_metrics})

,log_reg,random_forest
service_a,0.88,0.87
service_b,0.78,0.77
service_c,0.71,0.69
service_d,0.96,0.93
service_e,0.75,0.75
service_f,0.96,0.95
service_g,0.76,0.76
service_h,0.79,0.78
service_i,0.75,0.71
service_j,0.68,0.72


LogReg справляется немного лучше)

### У нас сильный дисбаланс классов, так что давай еще померим prec, recall, f1

In [16]:
# с параметрами играться не будем
clf = RandomForestClassifier(
    random_state=seed,
)
clf.fit(train_features, train_targets)
valid_preds = clf.predict(valid_features)


print(
    metrics.classification_report(
        valid_targets, valid_preds, target_names=train_labels_df.columns
    )
)

              precision    recall  f1-score   support

   service_a       0.84      0.71      0.77      1148
   service_b       0.62      0.26      0.36       815
   service_c       0.54      0.04      0.07       664
   service_d       0.00      0.00      0.00        40
   service_e       0.00      0.00      0.00       130
   service_f       0.00      0.00      0.00        67
   service_g       0.00      0.00      0.00       131
   service_h       0.78      0.32      0.45       724
   service_i       0.00      0.00      0.00        50
   service_j       0.88      0.99      0.93      2059
   service_k       0.81      0.99      0.89      1885
   service_l       1.00      0.03      0.07       262
   service_m       1.00      0.01      0.02       212
   service_n       0.58      0.02      0.03       399

   micro avg       0.82      0.61      0.70      8586
   macro avg       0.50      0.24      0.26      8586
weighted avg       0.75      0.61      0.60      8586
 samples avg       0.83   

/Users/n.v.barsukov2/Documents/code/tinkoff/ds_problem/.venv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/n.v.barsukov2/Documents/code/tinkoff/ds_problem/.venv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


#### Как видим, classification report не считается, потому что модель по некоторым лейблам модель выдает сплошные нули

In [17]:
valid_targets.iloc[:, 5].value_counts()

0    2333
1      67
Name: service_f, dtype: int64

Идеальный классификатор должен предсказать 67 единичек

In [32]:
valid_preds = np.transpose(
    [y_pred[:, 1] for y_pred in clf.predict_proba(valid_features)]
)

print("preds: ", valid_preds[:, 5])
print("top prob: ", np.sort(valid_preds[:, 5])[-5:])
print("number of predicted 1s: ", clf.predict(valid_features)[:, 5].sum())

preds:  [0.03 0.03 0.03 ... 0.03 0.03 0.03]
top prob:  [0.03 0.03 0.03 0.03 0.03]
number of predicted 1s:  0


### Откалибруем

In [33]:
from sklearn.calibration import CalibratedClassifierCV

# we need MultiOutputClassifier becase CalibratedClassifierCV only outputs one label
calibrated_clf = MultiOutputClassifier(
    CalibratedClassifierCV(
        RandomForestClassifier(
            random_state=seed,
        ),
        cv=3,
    )
)
calibrated_clf.fit(train_features, train_targets)

valid_preds = np.transpose(
    [y_pred[:, 1] for y_pred in calibrated_clf.predict_proba(valid_features)]
)

calibrated_metrics = pd.Series(
    index=train_labels_df.columns,
    data=metrics.roc_auc_score(
        valid_targets,
        valid_preds,
        average=None,
    ),
)

In [20]:
pd.DataFrame(
    {
        "log_reg": lr_metrics,
        "random_forest": rf_metrics,
        "calibrated_rf": calibrated_metrics,
    }
)

,log_reg,random_forest,calibrated_rf
service_a,0.88,0.87,0.88
service_b,0.78,0.77,0.78
service_c,0.71,0.69,0.70
service_d,0.96,0.93,0.93
service_e,0.75,0.75,0.75
service_f,0.96,0.95,0.96
service_g,0.76,0.76,0.77
service_h,0.79,0.78,0.79
service_i,0.75,0.71,0.71
service_j,0.68,0.72,0.71


Выдает примерно такие же метрики, но даже немного лучше

Пройдем заново предыдущий тест — сколько предскажет единичек?

In [34]:
print("preds: ", valid_preds[:, 5])
print("top prob: ", np.sort(valid_preds[:, 5])[-5:])
print("number of predicted 1s: ", calibrated_clf.predict(valid_features)[:, 5].sum())

preds:  [0.03 0.01 0.01 ... 0.01 0.01 0.01]
top prob:  [0.65 0.67 0.71 0.73 0.87]
number of predicted 1s:  20


уже лучше

In [22]:
print(
    metrics.classification_report(
        valid_targets,
        calibrated_clf.predict(valid_features),
        target_names=train_labels_df.columns,
    )
)

              precision    recall  f1-score   support

   service_a       0.85      0.73      0.78      1148
   service_b       0.64      0.34      0.45       815
   service_c       0.60      0.03      0.06       664
   service_d       0.67      0.05      0.09        40
   service_e       1.00      0.01      0.02       130
   service_f       0.40      0.12      0.18        67
   service_g       0.00      0.00      0.00       131
   service_h       0.72      0.39      0.51       724
   service_i       0.00      0.00      0.00        50
   service_j       0.88      0.99      0.93      2059
   service_k       0.81      0.99      0.89      1885
   service_l       0.71      0.20      0.31       262
   service_m       0.66      0.17      0.28       212
   service_n       0.50      0.17      0.25       399

   micro avg       0.81      0.64      0.72      8586
   macro avg       0.60      0.30      0.34      8586
weighted avg       0.75      0.64      0.64      8586
 samples avg       0.82   

/Users/n.v.barsukov2/Documents/code/tinkoff/ds_problem/.venv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/n.v.barsukov2/Documents/code/tinkoff/ds_problem/.venv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Все равно на двух лейблах выдает 0 единичек

#### Logreg дает более адекватный classification report

что ожидаемо, потому что он оптимизирует LogLoss — вероятность класса

In [23]:
# с параметрами играться не будем
inner_clf = LogisticRegression(solver="liblinear", random_state=seed)
clf = MultiOutputClassifier(inner_clf).fit(train_features, train_targets)
valid_preds = clf.predict(valid_features)


print(
    metrics.classification_report(
        valid_targets, valid_preds, target_names=train_labels_df.columns
    )
)

              precision    recall  f1-score   support

   service_a       0.88      0.70      0.78      1148
   service_b       0.65      0.53      0.58       815
   service_c       0.50      0.14      0.22       664
   service_d       0.38      0.15      0.21        40
   service_e       0.15      0.02      0.03       130
   service_f       0.24      0.15      0.19        67
   service_g       0.15      0.02      0.03       131
   service_h       0.71      0.42      0.53       724
   service_i       0.00      0.00      0.00        50
   service_j       0.88      0.98      0.93      2059
   service_k       0.81      0.96      0.88      1885
   service_l       0.63      0.25      0.36       262
   service_m       0.51      0.24      0.32       212
   service_n       0.41      0.21      0.28       399

   micro avg       0.79      0.66      0.72      8586
   macro avg       0.49      0.34      0.38      8586
weighted avg       0.73      0.66      0.67      8586
 samples avg       0.80   

/Users/n.v.barsukov2/Documents/code/tinkoff/ds_problem/.venv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## померим на целевой метрике logloss

In [24]:
logloss_metrics = pd.DataFrame(index=train_labels_df.columns)

In [25]:
# с параметрами играться не будем
inner_clf = LogisticRegression(solver="liblinear", random_state=seed)
clf = MultiOutputClassifier(inner_clf).fit(train_features, train_targets)

valid_preds = clf.predict_proba(valid_features)
valid_preds = np.transpose([y_pred[:, 1] for y_pred in valid_preds])

logloss_metrics["log_reg"] = [
    round(
        metrics.log_loss(valid_targets.iloc[:, label_id], valid_preds[:, label_id]),
        2,
    )
    for label_id, label in enumerate(train_labels_df.columns)
]

In [26]:
clf = RandomForestClassifier(
    random_state=seed,
)
clf.fit(train_features, train_targets)

valid_preds = clf.predict_proba(valid_features)
valid_preds = np.transpose([y_pred[:, 1] for y_pred in valid_preds])

logloss_metrics["random_forest"] = [
    round(
        metrics.log_loss(valid_targets.iloc[:, label_id], valid_preds[:, label_id]),
        2,
    )
    for label_id, label in enumerate(train_labels_df.columns)
]

In [27]:
clf = MultiOutputClassifier(
    CalibratedClassifierCV(
        RandomForestClassifier(
            random_state=seed,
        ),
        cv=3,
    )
)
clf.fit(train_features, train_targets)

valid_preds = clf.predict_proba(valid_features)
valid_preds = np.transpose([y_pred[:, 1] for y_pred in valid_preds])

logloss_metrics["rf_calibrated"] = [
    round(
        metrics.log_loss(valid_targets.iloc[:, label_id], valid_preds[:, label_id]),
        2,
    )
    for label_id, label in enumerate(train_labels_df.columns)
]

In [28]:
logloss_metrics

,log_reg,random_forest,rf_calibrated
service_a,0.41,0.48,0.43
service_b,0.53,0.55,0.54
service_c,0.54,0.54,0.54
service_d,0.05,0.07,0.05
service_e,0.20,0.20,0.19
service_f,0.08,0.08,0.08
service_g,0.20,0.19,0.19
service_h,0.48,0.50,0.49
service_i,0.10,0.15,0.10
service_j,0.37,0.36,0.35


Ура, я не тупой, калибровка и правда уменьшает логлосс

### Dummy: сравнимся с совсем тупым бейзлайном, который игнорирует входные фичи

In [29]:
from sklearn.dummy import DummyClassifier

clf = DummyClassifier(random_state=seed)
clf.fit(train_features, train_targets)

valid_preds = clf.predict_proba(valid_features)
valid_preds = np.transpose([y_pred[:, 1] for y_pred in valid_preds])

logloss_metrics["dummy"] = [
    round(
        metrics.log_loss(valid_targets.iloc[:, label_id], valid_preds[:, label_id]),
        2,
    )
    for label_id, label in enumerate(train_labels_df.columns)
]

In [30]:
logloss_metrics

,log_reg,random_forest,rf_calibrated,dummy
service_a,0.41,0.48,0.43,0.69
service_b,0.53,0.55,0.54,0.64
service_c,0.54,0.54,0.54,0.59
service_d,0.05,0.07,0.05,0.08
service_e,0.20,0.20,0.19,0.21
service_f,0.08,0.08,0.08,0.13
service_g,0.20,0.19,0.19,0.21
service_h,0.48,0.50,0.49,0.61
service_i,0.10,0.15,0.10,0.10
service_j,0.37,0.36,0.35,0.41


In [31]:
mean_metrics = logloss_metrics.mean()
mean_metrics

log_reg          0.30
random_forest    0.32
rf_calibrated    0.30
dummy            0.38
dtype: float64

# Вывод из бейзлана

Я не знаю целевую задачу, но обычно roc_auc 0.7+ хорош. В измерениях логлосса я тоже не особо понимаю, какая метрика хорошая.

Дамми алгоритм (не смотрящий на исходные фичи) справляется почти так же хорошо, как мои алгоритмы, так что бейзлан не самый сильный

Для обучения модели на этих фичах вполне хватает логрега